# Spracovanie dát

Tento notebook slúži na kontrolu kvality raw dát a ich predspracovanie.

In [232]:
import pandas as pd

In [233]:
cycle_data_raw = pd.read_csv('../Data/cyklotrasy_data_bratislava_raw.csv')
weather_data_raw = pd.read_csv('../Data/pocasie_data_raw.csv')

## Kvalita

In [234]:
# Skrátenie cyklo dát na časový rozsah počasia
cycle_data_raw["datetime"] = (
    pd.to_datetime(cycle_data_raw["attributes.DATUM_A_CAS"], utc=True)
      .dt.tz_convert("Europe/Bratislava")
      .dt.tz_localize(None)
)

weather_data_raw["date"] = pd.to_datetime(weather_data_raw["date"])

cycle_data_raw = cycle_data_raw[
    cycle_data_raw["datetime"].between(
        weather_data_raw["date"].min(),
        weather_data_raw["date"].max() + pd.Timedelta(days=0.9999)
    )
].copy()

In [235]:
display(cycle_data_raw.count())
display(weather_data_raw.count())

attributes.NAZOV              450301
attributes.ZEMEPISNA_SIRKA    450301
attributes.ZEMEPISNA_DLZKA    450301
attributes.SMER_DO            450301
attributes.SMER_Z             450301
attributes.DATUM_A_CAS        450301
attributes.POCET_DO           450301
attributes.POCET_Z            450301
attributes.ObjectId           450301
datetime                      450301
dtype: int64

date    1085
tavg    1085
tmin    1085
tmax    1085
prcp    1084
snow       0
wdir       0
wspd    1085
wpgt    1085
pres    1085
tsun     748
dtype: int64

In [236]:
cycle_data_raw.describe()

,attributes.ZEMEPISNA_SIRKA,attributes.ZEMEPISNA_DLZKA,attributes.POCET_DO,attributes.POCET_Z,attributes.ObjectId,datetime
count,450301.000000,450301.000000,450301.000000,450301.00000,450301.000000,450301
mean,48.149043,17.092158,12.107348,7.35377,318598.803971,2024-08-17 03:39:18.950124544
min,48.110442,16.969000,0.000000,0.00000,1.000000,2023-01-01 00:00:00
25%,48.136692,17.073272,0.000000,0.00000,115496.000000,2023-12-08 04:00:00
50%,48.140819,17.116058,2.000000,1.00000,233859.000000,2024-09-01 06:00:00
75%,48.162144,17.127928,14.000000,6.00000,580864.000000,2025-05-10 01:00:00
max,48.210650,17.138556,583.000000,645.00000,693456.000000,2025-12-20 23:00:00
std,0.025676,0.052929,23.829372,19.27489,232033.702638,NaN


In [237]:
weather_data_raw.describe()

,date,tavg,tmin,tmax,prcp,snow,wdir,wspd,wpgt,pres,tsun
count,1085,1085.000000,1085.000000,1085.000000,1084.000000,0.0,0.0,1085.000000,1085.000000,1085.000000,748.000000
mean,2024-06-26 00:00:00,12.521935,8.181659,16.956866,1.666697,NaN,NaN,12.416313,30.272258,1016.847926,267.161765
min,2023-01-01 00:00:00,-5.700000,-9.100000,-3.400000,0.000000,NaN,NaN,1.900000,11.100000,990.100000,0.000000
25%,2023-09-29 00:00:00,5.700000,2.000000,9.200000,0.000000,NaN,NaN,8.300000,24.100000,1011.700000,80.000000
50%,2024-06-26 00:00:00,12.300000,8.200000,16.600000,0.000000,NaN,NaN,11.400000,29.600000,1016.400000,237.000000
75%,2025-03-24 00:00:00,19.200000,14.100000,24.700000,1.025000,NaN,NaN,15.500000,35.200000,1021.800000,419.250000
max,2025-12-20 00:00:00,28.900000,23.900000,36.000000,37.300000,NaN,NaN,34.400000,72.200000,1043.300000,792.000000
std,NaN,8.104544,7.340852,9.297833,4.167819,NaN,NaN,5.387349,8.845390,8.208956,211.413720


In [238]:
print('Null values in cycle path data:', cycle_data_raw.isnull().sum())
print('Null values in weather data:', weather_data_raw.isnull().sum())

Null values in cycle path data: attributes.NAZOV              0
attributes.ZEMEPISNA_SIRKA    0
attributes.ZEMEPISNA_DLZKA    0
attributes.SMER_DO            0
attributes.SMER_Z             0
attributes.DATUM_A_CAS        0
attributes.POCET_DO           0
attributes.POCET_Z            0
attributes.ObjectId           0
datetime                      0
dtype: int64
Null values in weather data: date       0
tavg       0
tmin       0
tmax       0
prcp       1
snow    1085
wdir    1085
wspd       0
wpgt       0
pres       0
tsun     337
dtype: int64


In [239]:
dupes_cycle = cycle_data_raw.duplicated(subset=["attributes.ObjectId"]).sum()
print("Duplicated cycle path rows:", dupes_cycle)

dupes_weather = weather_data_raw.duplicated(subset=["date"]).sum()
print("Duplicated weather rows:", dupes_weather)

Duplicated cycle path rows: 0
Duplicated weather rows: 0


## Predspracovanie

### Cyklotrasy

In [240]:
# Premenovanie stĺpcov z 'attributes.*' na samotné názvy atribútov a vynechanie ObjectId a datum_a_cas (už máme 'date' ktorý je v správnej časovej zóne voči počasiu)
cycle_data = cycle_data_raw.rename(columns={
    'attributes.NAZOV': 'nazov',
    'attributes.ZEMEPISNA_SIRKA': 'zemepisna_sirka',
    'attributes.ZEMEPISNA_DLZKA': 'zemepisna_dlzka',
    'attributes.SMER_DO': 'smer_do',
    'attributes.SMER_Z': 'smer_z',
    'attributes.POCET_DO': 'pocet_do',
    'attributes.POCET_Z': 'pocet_z'})

cycle_data = cycle_data.drop(columns=['attributes.ObjectId', 'attributes.DATUM_A_CAS'])

In [241]:
def season_from_month(m):
    if m in [12, 1, 2]: return "winter"
    if m in [3, 4, 5]: return "spring"
    if m in [6, 7, 8]: return "summer"
    return "autumn"

In [242]:
# Vytvorenie nových stĺpcov pre analýzu
cycle_data["pocet_total"] = cycle_data["pocet_do"] + cycle_data["pocet_z"]
cycle_data["date"] = cycle_data_raw["datetime"].dt.normalize()
cycle_data["weekday"] = cycle_data["datetime"].dt.weekday
cycle_data["is_weekend"] = cycle_data["weekday"].isin([5, 6]).astype(int)
cycle_data["month"] = cycle_data["datetime"].dt.month
cycle_data["year"] = cycle_data["datetime"].dt.year
cycle_data["season"] = cycle_data["month"].apply(season_from_month)

### Počasie

In [243]:
# Vynechanie prázdnych stĺpcov
weather_data = weather_data_raw.drop(columns=['snow', 'wdir'])

# Doplnenie 0 pre chýbajúcu hodnotu zrážok - 1 riadok
weather_data['prcp'] = weather_data['prcp'].fillna(0)

## Spojenie datasetov

In [244]:
# date v cycle je aj s časom, v weather je len dátum, preto join na základe dátumu
data = cycle_data.merge(weather_data, on='date', how='left')

print(data.shape)
print(data["tavg"].isna().sum())

(450301, 23)
0


In [245]:
data.describe()

,zemepisna_sirka,zemepisna_dlzka,pocet_do,pocet_z,datetime,pocet_total,date,weekday,is_weekend,month,year,tavg,tmin,tmax,prcp,wspd,wpgt,pres,tsun
count,450301.000000,450301.000000,450301.000000,450301.00000,450301,450301.000000,450301,450301.000000,450301.000000,450301.000000,450301.000000,450301.000000,450301.000000,450301.000000,450301.000000,450301.000000,450301.000000,450301.000000,333433.000000
mean,48.149043,17.092158,12.107348,7.35377,2024-08-17 03:39:18.950124544,19.461118,2024-08-16 16:09:14.381180160,3.000486,0.285673,6.635426,2024.117162,12.863284,8.478058,17.350617,1.659009,12.377760,30.165796,1016.761736,270.738214
min,48.110442,16.969000,0.000000,0.00000,2023-01-01 00:00:00,0.000000,2023-01-01 00:00:00,0.000000,0.000000,1.000000,2023.000000,-5.700000,-9.100000,-3.400000,0.000000,1.900000,11.100000,990.100000,0.000000
25%,48.136692,17.073272,0.000000,0.00000,2023-12-08 04:00:00,0.000000,2023-12-08 00:00:00,1.000000,0.000000,4.000000,2023.000000,6.100000,2.500000,10.100000,0.000000,8.300000,24.100000,1011.700000,82.000000
50%,48.140819,17.116058,2.000000,1.00000,2024-09-01 06:00:00,4.000000,2024-09-01 00:00:00,3.000000,0.000000,7.000000,2024.000000,12.600000,8.800000,17.300000,0.000000,11.500000,29.600000,1016.200000,242.000000
75%,48.162144,17.127928,14.000000,6.00000,2025-05-10 01:00:00,23.000000,2025-05-10 00:00:00,5.000000,1.000000,10.000000,2025.000000,19.400000,14.300000,24.900000,0.900000,15.500000,35.200000,1021.700000,423.000000
max,48.210650,17.138556,583.000000,645.00000,2025-12-20 23:00:00,852.000000,2025-12-20 00:00:00,6.000000,1.000000,12.000000,2025.000000,28.900000,23.900000,36.000000,37.300000,34.400000,72.200000,1043.300000,792.000000
std,0.025676,0.052929,23.829372,19.27489,NaN,38.803733,NaN,1.999122,0.451735,3.337959,0.802385,8.010492,7.251459,9.210947,4.188012,5.294897,8.692311,7.978726,212.056414


In [246]:
data.isnull().sum()


nazov                   0
zemepisna_sirka         0
zemepisna_dlzka         0
smer_do                 0
smer_z                  0
pocet_do                0
pocet_z                 0
datetime                0
pocet_total             0
date                    0
weekday                 0
is_weekend              0
month                   0
year                    0
season                  0
tavg                    0
tmin                    0
tmax                    0
prcp                    0
wspd                    0
wpgt                    0
pres                    0
tsun               116868
dtype: int64

In [247]:
# Uloženie finálneho datasetu
data.to_csv('../Data/final_data.csv', index=False)